# Token classification

In [1]:
import transformers
print(f"Transformers version: {transformers.__version__}")


/root/miniconda3/lib/python3.12/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Transformers version: 4.56.0


In [2]:
print(f"Transformers version: {transformers.__file__}")
print(f"Transformers version: {transformers.__name__}")
print(f"Transformers version: {transformers.__path__}")

Transformers version: /root/miniconda3/lib/python3.12/site-packages/transformers/__init__.py
Transformers version: transformers
Transformers version: ['/root/miniconda3/lib/python3.12/site-packages/transformers']


In [3]:
import os

# 设置代理
os.environ['http_proxy'] = 'http://127.0.0.1:7893'
os.environ['https_proxy'] = 'http://127.0.0.1:7893'
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7893'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7893'
os.environ['no_proxy'] = '127.0.0.1,localhost'
os.environ['NO_PROXY'] = '127.0.0.1,localhost'

In [4]:
# 如果上面的方法不行，可以使用 WikiANN/WikiNER 数据集替代
# 这是一个多语言的命名实体识别数据集
from datasets import load_dataset

# 加载英文版本的 WikiANN 数据集
raw_datasets = load_dataset("wikiann", "en")

In [5]:
raw_datasets

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})

In [6]:
raw_datasets["train"][0]["tokens"]

['R.H.',
 'Saunders',
 '(',
 'St.',
 'Lawrence',
 'River',
 ')',
 '(',
 '968',
 'MW',
 ')']

In [7]:
raw_datasets["train"][0]["ner_tags"]

[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]

In [8]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))

In [9]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [10]:
words = raw_datasets["train"][2]["tokens"]
labels = raw_datasets["train"][2]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Karl  Ove   Knausgård ( born 1968 ) 
B-PER I-PER I-PER     O O    O    O 


In [11]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
tokenizer.is_fast

True

In [13]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'R',
 '.',
 'H',
 '.',
 'Saunders',
 '(',
 'St',
 '.',
 'Lawrence',
 'River',
 ')',
 '(',
 '96',
 '##8',
 'MW',
 ')',
 '[SEP]']

In [14]:
inputs.word_ids()

[None, 0, 0, 0, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, None]

In [15]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [16]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]
[-100, 3, 4, 4, 4, 4, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, -100]


In [17]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [18]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [19]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    4,    4,    4,    4,    0,    3,    4,    4,    4,    0,
            0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    1,    2,    2,    2,    0,    0,    0,
         -100, -100, -100, -100, -100, -100]])

In [21]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 4, 4, 4, 4, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, -100]


In [22]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [23]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [24]:
import evaluate

metric = evaluate.load("seqeval")

In [25]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O']

In [26]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'ORG': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(2)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(1.0),
 'overall_f1': np.float64(1.0),
 'overall_accuracy': 1.0}

In [27]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [28]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [29]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
model.config.num_labels

7

In [ ]:
from huggingface_hub import login

hf_token = "xxx"
login(hf_token)

In [34]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="bert-finetuned-ner",  # 明确指定为 output_dir
    eval_strategy="epoch",  # 改为 eval_strategy（简写）
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    per_device_train_batch_size=16,  # 建议添加
    per_device_eval_batch_size=16,   # 建议添加
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


* Trackio project initialized: huggingface
* Trackio metrics logged to: /root/autodl-tmp/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "huggingface"
* or by running in Python: trackio.show(project="huggingface")


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.306600,0.263642,0.782274,0.818606,0.800028,0.918901
2,0.205900,0.256573,0.795978,0.833734,0.814418,0.924188
3,0.145500,0.277656,0.809213,0.840732,0.824671,0.924977


* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)


TrainOutput(global_step=3750, training_loss=0.24561657104492188, metrics={'train_runtime': 221.3166, 'train_samples_per_second': 271.105, 'train_steps_per_second': 16.944, 'total_flos': 959590390953216.0, 'train_loss': 0.24561657104492188, 'epoch': 3.0})

In [35]:
trainer.push_to_hub(commit_message="Training complete")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ert-finetuned-ner/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...ntainer-8550119d52-9fdc5a82.56763.0: 100%|##########| 8.53kB / 8.53kB            

  ...ert-finetuned-ner/model.safetensors:  10%|9         | 41.9MB /  431MB            

CommitInfo(commit_url='https://huggingface.co/yiwenX/bert-finetuned-ner/commit/3e2586c764ea91576f991d126208106502e0f490', commit_message='Training complete', commit_description='', oid='3e2586c764ea91576f991d126208106502e0f490', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yiwenX/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='yiwenX/bert-finetuned-ner'), pr_revision=None, pr_num=None)

In [37]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [38]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [40]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [41]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [45]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner"
repo_name = get_full_repo_name(model_name)
repo_name

'yiwenX/bert-finetuned-ner'

In [ ]:
# 方法1：直接保存模型到本地（推荐）
output_dir = "bert-finetuned-ner-accelerate"

# 如果要推送到Hub，使用新的API
from huggingface_hub import create_repo

# 创建仓库（如果不存在）
try:
    create_repo(repo_name, exist_ok=True)
    print(f"仓库 {repo_name} 已准备好")
except Exception as e:
    print(f"创建仓库时出错: {e}")
    print("将只在本地保存模型")

In [47]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [49]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/7500 [00:00<?, ?it/s]

epoch 0: {'precision': np.float64(0.8329563127385834), 'recall': np.float64(0.7980358956992889), 'f1': np.float64(0.8151222717996611), 'accuracy': 0.9263528854836023}
epoch 1: {'precision': np.float64(0.8393185352749893), 'recall': np.float64(0.8013633909287257), 'f1': np.float64(0.8199019404737241), 'accuracy': 0.9271243348440883}
epoch 2: {'precision': np.float64(0.8393185352749893), 'recall': np.float64(0.8013633909287257), 'f1': np.float64(0.8199019404737241), 'accuracy': 0.9271243348440883}
